In [ ]:
!pip install requests
!pip install datasets
!pip install --upgrade accelerate
!pip install --upgrade transformers torch
!pip install --upgrade torchvision

In [ ]:
!pip install -U bitsandbytes

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import transformers
import torch
from datasets import load_dataset
import json
import os
import time

# Adatkészlet betöltése
dataset = load_dataset('Bazsalanszky/alpaca-cleaned-gemini-hun')

# Modell betöltése és inicializálása
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# BFLOAT16

In [ ]:
import transformers
import torch
from datasets import load_dataset
import json
import os
import time

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

# Analízis végrehajtása a LLM modellel
def analyze_interaction(input_text, response_text):
    messages = [
        {"role": "system", "content": "Egy segéd vagy, aki a user - assistant interakciót elemzi. Az aszisztant válasza mennyire felelt meg a felhaszálói kérésnek 1-10 közt. Csak egy számot írj, semmi mást!"},
        {"role": "user", "content": f"Felhasználói utasítás:\n{input_text}\nAssistant válasz:\n{response_text}"}
    ]

    outputs = pipeline(messages, max_new_tokens=10)
    assistant_response = outputs[0]['generated_text'][-1]['content'].strip()
    return assistant_response

def add_analysis(example):
    analysis = analyze_interaction(example['input'], example['response'])
    example['analize'] = analysis
    return example

def process_chunk(chunk, chunk_id, num_chunks, chunk_dir):
    print(f"Processing chunk {chunk_id + 1}/{num_chunks}...")
    chunk = chunk.map(add_analysis)
    chunk_file = os.path.join(chunk_dir, f'chunk_{chunk_id}.jsonl')
    with open(chunk_file, 'w', encoding='utf-8') as f:
        for item in chunk:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    print(f"Chunk {chunk_id} saved to {chunk_file}.")
    print(f"Progress: {((chunk_id + 1) / num_chunks) * 100:.2f}%")

def get_last_processed_chunk(chunk_dir):
    chunk_files = [f for f in os.listdir(chunk_dir) if f.startswith('chunk_') and f.endswith('.jsonl')]
    if not chunk_files:
        return -1
    chunk_ids = [int(f.split('_')[1].split('.')[0]) for f in chunk_files]
    return max(chunk_ids)

# Ellenőrizzük az utolsó feldolgozott chunk-ot
chunk_dir = 'chunks'
if not os.path.exists(chunk_dir):
    os.makedirs(chunk_dir)
last_processed_chunk = get_last_processed_chunk(chunk_dir)

# Adatok 1000 soros darabokra bontása és feldolgozása
chunk_size = 1000
num_chunks = (len(dataset['train']) + chunk_size - 1) // chunk_size

# Összes feldolgozási idő mérése
start_time = time.time()

for chunk_id in range(last_processed_chunk + 1, num_chunks):
    start_idx = chunk_id * chunk_size
    end_idx = min(start_idx + chunk_size, len(dataset['train']))
    chunk = dataset['train'].select(range(start_idx, end_idx))
    process_chunk(chunk, chunk_id, num_chunks, chunk_dir)

# Az összesített eredmények összegyűjtése
def aggregate_results(num_chunks, chunk_dir):
    all_data = []
    for chunk_id in range(num_chunks):
        chunk_file = os.path.join(chunk_dir, f'chunk_{chunk_id}.jsonl')
        if os.path.exists(chunk_file):
            with open(chunk_file, 'r', encoding='utf-8') as f:
                for line in f:
                    all_data.append(json.loads(line))
    return all_data

all_results = aggregate_results(num_chunks, chunk_dir)

# Összesített eredmények mentése
with open('data_with_analysis.jsonl', 'w', encoding='utf-8') as f:
    for item in all_results:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

end_time = time.time()
total_time = end_time - start_time
print(f"All chunks have been processed and aggregated in {total_time:.2f} seconds.")


# FLOAT16

In [ ]:
import transformers
import torch
from datasets import load_dataset
import json
import os
import time

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.float16},
    device_map="auto",
)

# Analízis végrehajtása a LLM modellel
def analyze_interaction(input_text, response_text, instruction_text):
    if input_text:
        prompt = f"Felhasználói utasítás:\n{input_text}\n\n{instruction_text}\nAssistant válasz:\n{response_text}"
    else:
        prompt = f"Felhasználói utasítás:\n{instruction_text}\nAssistant válasz:\n{response_text}"

    messages = [
        {"role": "system", "content": "Egy segéd vagy, aki a user - assistant interakciót elemzi. Az aszisztant válasza mennyire felelt meg a felhaszálói kérésnek 1-10 közt. Csak egy számot írj, semmi mást!"},
        {"role": "user", "content": prompt}
    ]

    outputs = pipeline(messages, max_new_tokens=10)
    assistant_response = outputs[0]['generated_text'][-1]['content'].strip()
    return assistant_response

def add_analysis(example):
    input_text = example['input']
    instruction_text = example['instruction']
    response_text = example['response']
    analysis = analyze_interaction(input_text, response_text, instruction_text)
    example['analize'] = analysis
    return example

def process_chunk(chunk, chunk_id, num_chunks, chunk_dir):
    print(f"Processing chunk {chunk_id + 1}/{num_chunks}...")
    chunk = chunk.map(add_analysis)
    chunk_file = os.path.join(chunk_dir, f'chunk_{chunk_id}.jsonl')
    with open(chunk_file, 'w', encoding='utf-8') as f:
        for item in chunk:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    print(f"Chunk {chunk_id} saved to {chunk_file}.")
    print(f"Progress: {((chunk_id + 1) / num_chunks) * 100:.2f}%")

def get_last_processed_chunk(chunk_dir):
    chunk_files = [f for f in os.listdir(chunk_dir) if f.startswith('chunk_') and f.endswith('.jsonl')]
    if not chunk_files:
        return -1
    chunk_ids = [int(f.split('_')[1].split('.')[0]) for f in chunk_files]
    return max(chunk_ids)

# Ellenőrizzük az utolsó feldolgozott chunk-ot
chunk_dir = 'chunks'
if not os.path.exists(chunk_dir):
    os.makedirs(chunk_dir)
last_processed_chunk = get_last_processed_chunk(chunk_dir)

# Adatok 1000 soros darabokra bontása és feldolgozása
chunk_size = 1000
num_chunks = (len(dataset['train']) + chunk_size - 1) // chunk_size

# Összes feldolgozási idő mérése
start_time = time.time()

for chunk_id in range(last_processed_chunk + 1, num_chunks):
    start_idx = chunk_id * chunk_size
    end_idx = min(start_idx + chunk_size, len(dataset['train']))
    chunk = dataset['train'].select(range(start_idx, end_idx))
    process_chunk(chunk, chunk_id, num_chunks, chunk_dir)

# Az összesített eredmények összegyűjtése
def aggregate_results(num_chunks, chunk_dir):
    all_data = []
    for chunk_id in range(num_chunks):
        chunk_file = os.path.join(chunk_dir, f'chunk_{chunk_id}.jsonl')
        if os.path.exists(chunk_file):
            with open(chunk_file, 'r', encoding='utf-8') as f:
                for line in f:
                    all_data.append(json.loads(line))
    return all_data

all_results = aggregate_results(num_chunks, chunk_dir)

# Összesített eredmények mentése
with open('data_with_analysis.jsonl', 'w', encoding='utf-8') as f:
    for item in all_results:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

end_time = time.time()
total_time = end_time - start_time
print(f"All chunks have been processed and aggregated in {total_time:.2f} seconds.")


# FLOAT16 faster

In [ ]:
import transformers
import torch
from datasets import load_dataset
import json
import os
import time

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.float16},
    device_map="auto",
)

# Analízis végrehajtása a LLM modellel
def analyze_interaction(input_text, response_text, instruction_text):
    if input_text:
        prompt = f"Felhasználói utasítás:\n{input_text}\n\n{instruction_text}\nAssistant válasz:\n{response_text}"
    else:
        prompt = f"Felhasználói utasítás:\n{instruction_text}\nAssistant válasz:\n{response_text}"

    messages = [
        {"role": "system", "content": "Egy segéd vagy, aki a user - assistant interakciót elemzi. Az aszisztant válasza mennyire felelt meg a felhaszálói kérésnek vagy kérdésnek 1-10 közt. Gondold át, légy alapos. Csak egy számot írj, semmi mást!"},
        {"role": "user", "content": prompt}
    ]

    outputs = pipeline(messages, max_new_tokens=10)
    assistant_response = outputs[0]['generated_text'][-1]['content'].strip()
    return assistant_response

def add_analysis(example):
    input_text = example['input']
    instruction_text = example['instruction']
    response_text = example['response']
    analysis = analyze_interaction(input_text, response_text, instruction_text)
    example['analize'] = analysis
    return example

def process_chunk(chunk, chunk_id, num_chunks, chunk_dir):
    print(f"Processing chunk {chunk_id + 1}/{num_chunks}...")
    chunk = chunk.map(add_analysis)
    chunk_file = os.path.join(chunk_dir, f'chunk_{chunk_id}.jsonl')
    with open(chunk_file, 'w', encoding='utf-8') as f:
        for item in chunk:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    print(f"Chunk {chunk_id} saved to {chunk_file}.")
    print(f"Progress: {((chunk_id + 1) / num_chunks) * 100:.2f}%")

def get_last_processed_chunk(chunk_dir):
    chunk_files = [f for f in os.listdir(chunk_dir) if f.startswith('chunk_') and f.endswith('.jsonl')]
    if not chunk_files:
        return -1
    chunk_ids = [int(f.split('_')[1].split('.')[0]) for f in chunk_files]
    return max(chunk_ids)

# Ellenőrizzük az utolsó feldolgozott chunk-ot
chunk_dir = 'chunks'
if not os.path.exists(chunk_dir):
    os.makedirs(chunk_dir)
last_processed_chunk = get_last_processed_chunk(chunk_dir)

# Adatok 1000 soros darabokra bontása és feldolgozása
chunk_size = 1000
num_chunks = (len(dataset['train']) + chunk_size - 1) // chunk_size

# Összes feldolgozási idő mérése
start_time = time.time()

for chunk_id in range(last_processed_chunk + 1, num_chunks):
    start_idx = chunk_id * chunk_size
    end_idx = min(start_idx + chunk_size, len(dataset['train']))
    chunk = dataset['train'].select(range(start_idx, end_idx))
    process_chunk(chunk, chunk_id, num_chunks, chunk_dir)

# Az összesített eredmények összegyűjtése
def aggregate_results(num_chunks, chunk_dir):
    all_data = []
    for chunk_id in range(num_chunks):
        chunk_file = os.path.join(chunk_dir, f'chunk_{chunk_id}.jsonl')
        if os.path.exists(chunk_file):
            with open(chunk_file, 'r', encoding='utf-8') as f:
                for line in f:
                    all_data.append(json.loads(line))
    return all_data

all_results = aggregate_results(num_chunks, chunk_dir)

# Összesített eredmények mentése
with open('data_with_analysis.jsonl', 'w', encoding='utf-8') as f:
    for item in all_results:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

end_time = time.time()
total_time = end_time - start_time
print(f"All chunks have been processed and aggregated in {total_time:.2f} seconds.")


# INT8

In [ ]:
import transformers
import torch
from datasets import load_dataset
import json
import os
import time
from transformers import AutoModelForCausalLM, AutoTokenizer

# Tokenizer betöltése
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Modell betöltése és kvantálása
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,  # INT8 kvantálás engedélyezése
    device_map="auto",
)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
)

# Analízis végrehajtása a LLM modellel
def analyze_interaction(input_text, response_text, instruction_text):
    if input_text:
        prompt = f"Felhasználói utasítás:\n{input_text}\n\n{instruction_text}\nAssistant válasz:\n{response_text}"
    else:
        prompt = f"Felhasználói utasítás:\n{instruction_text}\nAssistant válasz:\n{response_text}"

    messages = [
        {"role": "system", "content": "Egy segéd vagy, aki a user - assistant interakciót elemzi. Az aszisztant válasza mennyire felelt meg a felhaszálói kérésnek vagy kérdésnek 1-10 közt. Gondold át, légy alapos. Csak egy számot írj, semmi mást!"},
        {"role": "user", "content": prompt}
    ]

    outputs = pipeline(messages, max_new_tokens=10)
    assistant_response = outputs[0]['generated_text'][-1]['content'].strip()
    return assistant_response

def add_analysis(example):
    input_text = example['input']
    instruction_text = example['instruction']
    response_text = example['response']
    analysis = analyze_interaction(input_text, response_text, instruction_text)
    example['analize'] = analysis
    return example

def process_chunk(chunk, chunk_id, num_chunks, chunk_dir):
    print(f"Processing chunk {chunk_id + 1}/{num_chunks}...")
    chunk = chunk.map(add_analysis)
    chunk_file = os.path.join(chunk_dir, f'chunk_{chunk_id}.jsonl')
    with open(chunk_file, 'w', encoding='utf-8') as f:
        for item in chunk:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    print(f"Chunk {chunk_id} saved to {chunk_file}.")
    print(f"Progress: {((chunk_id + 1) / num_chunks) * 100:.2f}%")

def get_last_processed_chunk(chunk_dir):
    chunk_files = [f for f in os.listdir(chunk_dir) if f.startswith('chunk_') and f.endswith('.jsonl')]
    if not chunk_files:
        return -1
    chunk_ids = [int(f.split('_')[1].split('.')[0]) for f in chunk_files]
    return max(chunk_ids)

# Ellenőrizzük az utolsó feldolgozott chunk-ot
chunk_dir = 'chunks'
if not os.path.exists(chunk_dir):
    os.makedirs(chunk_dir)
last_processed_chunk = get_last_processed_chunk(chunk_dir)

# Adatok 1000 soros darabokra bontása és feldolgozása
chunk_size = 1000
num_chunks = (len(dataset['train']) + chunk_size - 1) // chunk_size

# Összes feldolgozási idő mérése
start_time = time.time()

for chunk_id in range(last_processed_chunk + 1, num_chunks):
    start_idx = chunk_id * chunk_size
    end_idx = min(start_idx + chunk_size, len(dataset['train']))
    chunk = dataset['train'].select(range(start_idx, end_idx))
    process_chunk(chunk, chunk_id, num_chunks, chunk_dir)

# Az összesített eredmények összegyűjtése
def aggregate_results(num_chunks, chunk_dir):
    all_data = []
    for chunk_id in range(num_chunks):
        chunk_file = os.path.join(chunk_dir, f'chunk_{chunk_id}.jsonl')
        if os.path.exists(chunk_file):
            with open(chunk_file, 'r', encoding='utf-8') as f:
                for line in f:
                    all_data.append(json.loads(line))
    return all_data

all_results = aggregate_results(num_chunks, chunk_dir)

# Összesített eredmények mentése
with open('data_with_analysis.jsonl', 'w', encoding='utf-8') as f:
    for item in all_results:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

end_time = time.time()
total_time = end_time - start_time
print(f"All chunks have been processed and aggregated in {total_time:.2f} seconds.")

# GGUF (not working)

In [ ]:
!wget https://huggingface.co/lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf?download=true

In [ ]:
!CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS" pip install llama-cpp-python

!pip install langchain

In [5]:
!python -V

Python 3.10.12


In [ ]:
!nvcc --version

In [ ]:
!set LLAMA_CUBLAS=1
!set CMAKE_ARGS=-DLLAMA_CUBLAS=on
!set FORCE_CMAKE=1
!python -m pip install llama-cpp-python==0.2.83 --prefer-binary --extra-index-url=https://github.com/jllllll/llama-cpp-python-cuBLAS-wheels/releases/download/wheels/llama_cpp_python_ggml-0.1.78+cu122-cp310-cp310-linux_x86_64.whl
!pip install torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 --index-url https://download.pytorch.org/whl/cu121

In [ ]:
!pip install -r gpu_requirements.txt

In [ ]:
!pip install --upgrade --force-reinstall  llama-cpp-python \
  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122

In [ ]:
import json
import os
import time
from datasets import load_dataset
from llama_cpp import Llama

# Modell betöltése és inicializálása gguf Q4_M kvantálással
model_path = "/content/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf"


llm = Llama(model_path=model_path, n_gpu_layers=-1)

# Analízis végrehajtása a LLM modellel
def analyze_interaction(input_text, response_text, instruction_text):
    if input_text:
        prompt = f"Felhasználói utasítás:\n{input_text}\n\n{instruction_text}\nAssistant válasz:\n{response_text}"
    else:
        prompt = f"Felhasználói utasítás:\n{instruction_text}\nAssistant válasz:\n{response_text}"

    full_prompt = f"Egy segéd vagy, aki a user - assistant interakciót elemzi. Az asszisztant válasza mennyire felelt meg a felhaszálói kérésnek, kérdésnek 1-10 közt. Csak egy számot írj, semmi mást!\n{prompt}"

    response = llm(full_prompt, max_tokens=10)
    assistant_response = response['choices'][0]['text'].strip()
    return assistant_response

def add_analysis(example):
    input_text = example['input']
    instruction_text = example['instruction']
    response_text = example['response']
    analysis = analyze_interaction(input_text, response_text, instruction_text)
    example['analize'] = analysis
    return example

def process_chunk(chunk, chunk_id, num_chunks, chunk_dir):
    print(f"Processing chunk {chunk_id + 1}/{num_chunks}...")
    chunk = chunk.map(add_analysis)
    chunk_file = os.path.join(chunk_dir, f'chunk_{chunk_id}.jsonl')
    with open(chunk_file, 'w', encoding='utf-8') as f:
        for item in chunk:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    print(f"Chunk {chunk_id} saved to {chunk_file}.")
    print(f"Progress: {((chunk_id + 1) / num_chunks) * 100:.2f}%")

def get_last_processed_chunk(chunk_dir):
    chunk_files = [f for f in os.listdir(chunk_dir) if f.startswith('chunk_') and f.endswith('.jsonl')]
    if not chunk_files:
        return -1
    chunk_ids = [int(f.split('_')[1].split('.')[0]) for f in chunk_files]
    return max(chunk_ids)

# Ellenőrizzük az utolsó feldolgozott chunk-ot
chunk_dir = 'chunks3'
if not os.path.exists(chunk_dir):
    os.makedirs(chunk_dir)
last_processed_chunk = get_last_processed_chunk(chunk_dir)

# Adatok 5000 soros darabokra bontása és feldolgozása
chunk_size = 5000
num_chunks = (len(dataset['train']) + chunk_size - 1) // chunk_size

# Összes feldolgozási idő mérése
start_time = time.time()

for chunk_id in range(last_processed_chunk + 1, num_chunks):
    start_idx = chunk_id * chunk_size
    end_idx = min(start_idx + chunk_size, len(dataset['train']))
    chunk = dataset['train'].select(range(start_idx, end_idx))
    process_chunk(chunk, chunk_id, num_chunks, chunk_dir)

# Az összesített eredmények összegyűjtése
def aggregate_results(num_chunks, chunk_dir):
    all_data = []
    for chunk_id in range(num_chunks):
        chunk_file = os.path.join(chunk_dir, f'chunk_{chunk_id}.jsonl')
        if os.path.exists(chunk_file):
            with open(chunk_file, 'r', encoding='utf-8') as f:
                for line in f:
                    all_data.append(json.loads(line))
    return all_data

all_results = aggregate_results(num_chunks, chunk_dir)

# Összesített eredmények mentése
with open('data_with_analysis.jsonl', 'w', encoding='utf-8') as f:
    for item in all_results:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

end_time = time.time()
total_time = end_time - start_time
print(f"All chunks have been processed and aggregated in {total_time:.2f} seconds.")


# Adatelemzés

In [6]:
import json

# Betöltjük a data_with_analysis.jsonl fájlt
input_file = 'data_with_analysis.jsonl'
output_file = 'filtered_data_with_analysis_8-10.jsonl'

filtered_data = []

with open(input_file, 'r', encoding='utf-8') as infile:
    for line in infile:
        data = json.loads(line)
        try:
            # Ellenőrizzük az értékelés értékét, és ha 8 vagy nagyobb, megtartjuk a sort
            if int(data['analize']) >= 8:
                filtered_data.append(data)
        except ValueError:
            # Ha az értékelés nem konvertálható számmá, kihagyjuk a sort
            continue

# Az eredményeket egy új fájlba mentjük
with open(output_file, 'w', encoding='utf-8') as outfile:
    for item in filtered_data:
        outfile.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"Filtered data saved to {output_file}.")


Filtered data saved to filtered_data_with_analysis_8-10.jsonl.


In [2]:
import json

# A jsonl fájl olvasása
input_file = 'data_with_analysis.jsonl'

# Külön fájlok meghatározása az 'analize' oszlop értékei szerint
output_files = {
    '1': 'output_1.jsonl',
    '2': 'output_2.jsonl',
    '3': 'output_3.jsonl',
    '4': 'output_4.jsonl',
    '5': 'output_5.jsonl',
    '6': 'output_6.jsonl',
    '7': 'output_7.jsonl',
    '8': 'output_8.jsonl',
    '9': 'output_9.jsonl',
    '10': 'output_10.jsonl',
    'non_numeric': 'output_non_numeric.jsonl'
}

# Üres fájlok létrehozása és megnyitása írásra
output_handlers = {key: open(output_files[key], 'w') for key in output_files}

with open(input_file, 'r') as infile:
    for line in infile:
        data = json.loads(line)
        analize_value = str(data.get('analize', 'non_numeric'))

        # Ellenőrzés, hogy az 'analize' érték numerikus-e és az 1-10 tartományba esik-e
        if analize_value.isdigit() and analize_value in output_handlers:
            output_handlers[analize_value].write(line)
        else:
            output_handlers['non_numeric'].write(line)

# Fájlok bezárása
for handler in output_handlers.values():
    handler.close()
